<a id="1000"></a>
## Indice parte 2:

- [NOTE](#100)
- [PARTE 1: Classe CorsConfig](#1)
- [PARTE 2: Implementazione Mail Service](#2)
- [PARTE 3: Utente modifica se stesso](#3)
- [PARTE 4: Gestione file(uploadAvatar)](#4)

<a id="100"></a>
### NOTE VARIE

- Nella prima parte del tutorial, abbiamo delineato la struttura del nostro progetto, gettando le basi per lo sviluppo. Nella seconda parte, ci concentreremo sullo sviluppo pratico, costruendo passo dopo passo le diverse classi e approfondendo la logica di programmazione dietro di esse. Questa fase è essenziale per comprendere come ogni classe contribuisca alle funzionalità e alle politiche del progetto, spiegando sia il "come" sia il "perché" delle nostre scelte di progettazione e codifica. In breve, questa sezione è un percorso guidato e dettagliato attraverso l'effettiva implementazione del progetto.
- Nelle Entity le PK sono annotate. Quando i repository chiamano le richerche per Id come existById non c'è bisogno di riportare il nome preciso dell'id della classe, in quanto l'annotazione rende chiara quale attributo è PK a prescindere dal suo nome.
- Quando usiamo alcune operazioni sui Set come set1.remove(set2), entrambi i set devono aver implementato il metodo equals(). E' importante che il metodo equals prenda in considerazione solo l'id altrimenti oggetti con lo stesso id ma stato diverso (Es. tag visibile/non visibile) verrebbero considerati come oggetti diversi.
- In genere non si fanno mai cancellazioni di record ma il record da "cancellare" viene semplicemente disabilitato da un campo di flag. 

<a id="1"></a>
### PARTE 1: Classe CorsConfig

Aggiungiamo la classe CorsCon
fig che si occuperà di gestire da quali origini possono essere mandate le richieste, quali sono gli header accettati e i metodi che possono essere chiamati sugli endpoint.
Annotazioni:
<span style="background-color:#00FFFF;">@Coguration</span><span style="background-color:#FFFF00;">(Classe)</span>

[Torna all'indice](#1000)

<a id="2"></a>
### PARTE 2: Implementazione Mail Service

Dopo il singIn l'utente non è ancora abilitato. Dovrà ricevere una mail con un link per l'abilitazione che faccia riferimento ad un metodo nel back end. L'url che l'utente riceverà e che richiamerà il back end dovrà contenere un riferimento all'utente(JWT).

1. Aggiunta properties file yaml
```yaml
jpa:
.
.
.
  mail:
    host:  smtp.gmail.com
    port: 587
    # solo username senza gmail.com
    username: ${CORSO_USER_GMAIL}
    password: ${CORSO_PASSWORD_GMAIL}
    properties:
      mail:
        smtp:
          auth: true
          starttls:
            required: true

#CUSTOM PROPERTIES
app:
  mail:
    sender: ${CORSO_MAIL_MITTENTE_GMAIL}
```
dove:
- CORSO_USER_GMAIL è il solo nome utente della mail senza il dominio
- CORSO_PASSWORD_GMAIL è il vlaore fornito da goolge per l'autenticazione in due passaggi
- CORSO_MAIL_MITTENTE_GMAIL è la mail completa dell'utente

2. In ("/service") creo ("/service/mail") e al suo interno le classi <span style="background-color:#FF00FF;">Mail e MailService</span>.<br>In MailService iniettiamo JavaMailSender, classu di utilità di JAva Spring per la gestione dell'invio di mail. Creiamo i metodi createMail(Sender(User), subject, object) e sendMail(Mail).

3. Creo in <span style="background-color:#FF00FF;">("controller")AuthenticationController e ("service")AuthenticationService</span> il metodo confirm(JWT) che abilita l'utente. Il metodo sarà chiamato cliccando sul link ricevuto via mail da''utente.<br> 
Abbiamo usato in questo caso l'annotazione <span style="background-color:#00FFFF;">@Transactional</span><span style="background-color:#FFFF00;">(Metodo)</span> che effettuerà una save dell'oggetto User solo nel caso in cui tutti i controlli vadano a buon fine. Se tutti i passaggi della transazione non sono rispettati viene effettuato un ROLL Back.

4. Modifico il metodo signUp nel AuthenticationService aggiungendo il codice per inviare la mail.

[Torna all'indice](#1000)

<a id="3"></a>
### PARTE 3: Utente modifica se stesso

Dobbiamo gestire la possibilità per l'utente di modificare il dati sul proprio profilo.

1. Creiamo <span style="background-color:#FF00FF;">("service/")UserService, ("/controller")UserController, ("dto/request") ChangePwdRequest - ChangeRoleRequest - NotifyMeRequest</span>.
2. In UserService e USerController creiamo i metodi: 
    - <span style="background-color:#FF00FF;">changePassword(ChangePwdRequest, User)</span>: LA politica è di inserire prima la vecchia pwd. Per controllare la password dovrò utilizzare la classe PasswordEncoder con il metodo .match(OldPwd,NewPwd). Ripetere la nuova pwd 2 volte e confrontare se i due inserimenti sono uguali.
   
    - <span style="background-color:#FF00FF;">hangeRole(ChangeRoleRequest, User)</span>: solo gli Admin possono cambiare i ruoli
    
    - <span style="background-color:#FF00FF;">modifyMe(ModifyMeRequest, User)</span>: 
        1. Voglio cambiare la mia mail ma la mail è già presente nel DB relativamente ad un altro utente
        2. Cambio tutti i campi tramite la mail.
        3. Se cambio la mail in maniera valida la mail dovrà essere nuovamente confermata. Fino alla conferma l'utente sarà disabilitato.

Tutti i metodi di modifica dell'utente richiedono di recuperare l'utente loggato. In spring Security l'utente loggato se implementa UserDetails viene salvato globalmente ed è possibile recuperarlo tramite l'annotazione
<span style="background-color:#00FFFF;">@AuthneticationPrincipal</span><span style="background-color:#FFFF00;">(Parametro)</span> su un oggetto USerDetails. Lo UserDetails conterrà tutti i campi dell'utente loggato e potrà essere "castato" a USer.

[Torna all'indice](#1000)

<a id="4"></a>
### PARTE 4: Gestione file(uploadAvatar)

Un avatar è costituito da una immagine. Quindi dovrò passare un file tramite una richiesta Http.<br>
In spring abbiamo una classe MultipartFile che permette la gestione di file inviati tramite richiesta Http in maniera generica a prescindere dalla loro estensione.
Se un endpoint utilizza l'annotazione <span style="background-color:#00FFFF;">@PostMapping</span><span style="background-color:#FFFF00;">(Metodo)</span> dobbiamo fare alcune precisazioni. Infatti un metodo ha sempre un oggetto "consumato" che in genere di default è un oggetto JSON. Nel caso in cui l'oggetto sia una immafine dobbiamo modificare nel seguente modo:
```java
 @PostMapping(value = "add-avatar", consumes = MediaType.MULTIPART_FORM_DATA_VALUE)
```

In pratica diciamo che il tipo di dato che l'endpoint si aspetta è un file Multipart.

1. Aggiungiamo alcune custom properties nel file .yaml:
```yaml
app:
  .
  . 
  .
  image:
    avatar:
      size: 5120
      height: 100
      width: 100
      extensions: gif,png,jpg,jpeg
```
Le custom properties sono accessibili tramite l'annotazione <span style="background-color:#00FFFF;">@Value</span><span style="background-color:#FFFF00;">(Attributo)</span>:
```java
    @Value("${app.image.avatar.size}")
    private long size;
```
2. Creiamo ("/service")ImageService dove avremo tutti i metodi di gestione dell'immagine:
    - checkSize(MultipartFile, size)
    - checkDimension(MultipartFile, width, height)
    - checkExtensions(MultipartFile, []allowedExtensions)
    - fromMultipartFileToBufferedImage(MultipartFile)
    Questi metodi necessitano di trasformare il nostro MultipartFile in un file BufferedImage che ci consente di avere i metodi di estrazione delle propietà dell'immagine che ci interessa controllare.
3. Modifichiamo ("/controller")UserController inserendo il metodo UploadAVatar(MultipartFile, UserDetails).
 
[Torna all'indice](#1000)